# Shallow Moment Tutorial (Simple)

## Imports

In [1]:
# | code-fold: true
# | code-summary: "Load packages"
# | output: false

import os
import numpy as np
import jax
from jax import numpy as jnp
import pytest
from types import SimpleNamespace
from sympy import cos, pi, Piecewise

from library.fvm.solver import HyperbolicSolver, Settings
from library.fvm.ode import RK1
import library.fvm.reconstruction as recon
import library.fvm.timestepping as timestepping
import library.fvm.flux as flux
import library.fvm.nonconservative_flux as nc_flux
from library.model.boundary_conditions import BoundaryCondition
from library.model.models.basisfunctions import Basisfunction, Legendre_shifted
from library.model.models.basismatrices import Basismatrices
from library.misc.misc import Zstruct

from library.model.models.shallow_moments import ShallowMoments2d
import library.model.initial_conditions as IC
import library.model.boundary_conditions as BC
import library.misc.io as io
from library.mesh.mesh import compute_derivatives
from tests.pdesoft import plots_paper
import library.postprocessing.visualization as visu


import library.mesh.mesh as petscMesh
import library.postprocessing.postprocessing as postprocessing
from library.mesh.mesh import convert_mesh_to_jax
import argparse

2025-08-07 07:55:59.784 | WARNING  | library.fvm.solver:<module>:24 - No module named 'precice'


## Model

In [2]:
level = 0
offset = 1+level
n_fields = 3 + 2 * level

settings = Settings(
    name="SME",
    model=Zstruct(
        level=level,
        parameters=Zstruct.from_dict({
        "g": 9.81,
        'ex': 0.,
        'ey': 0.,
        'ez': 1.,
        "C": 1.0,
        "nu": 0.000001,
        "lamda": 7,
        "rho": 1,
        "eta": 1,
        "c_slipmod": 1 / 7.0,
    }),
    ),
    solver=Zstruct(time_end=1.0, CFL=0.45),
    output=Zstruct(
        directory=f"outputs/junction_{level}", filename="SME", output_snapshots=30
    ),
)

2025-08-07 07:56:02.607 | WARNING  | library.misc.misc:__init__:153 - No 'clean_directory' attribute found in output Zstruct. Default: False


In [3]:
inflow_dict = { 
    0: lambda t, x, dx, q, qaux, p, n: Piecewise((0.1, t < 0.2),(q[0], True)),
    1: lambda t, x, dx, q, qaux, p, n: Piecewise((-0.3, t < 0.2),(-q[1], True)),
                }
inflow_dict.update({
    1+i: lambda t, x, dx, q, qaux, p, n: 0 for i in range(level)
})
inflow_dict.update({
    1+offset+i: lambda t, x, dx, q, qaux, p, n: 0 for i in range(level+1)
})

bcs = BC.BoundaryConditions(
    [
        BC.Lambda(physical_tag="inflow", prescribe_fields=inflow_dict),
        BC.Wall(physical_tag="wall"),
    ]
)

def custom_ic(x):
    Q = np.zeros(3 + 2 * level, dtype=float)
    Q[0] = 0.01
    return Q

ic = IC.UserFunction(custom_ic)

model = ShallowMoments2d(
    fields=3 + 2 * level,
    aux_fields=2,
    parameters=settings.model.parameters.as_dict(),
    boundary_conditions=bcs,
    initial_conditions=ic,
    settings={"friction": ["newtonian", "slip_mod"]},
    basis=Basismatrices(basis=Legendre_shifted(order=level+1)),
)

main_dir = os.getenv("SMS")
mesh = petscMesh.Mesh.from_gmsh(
    os.path.join(main_dir, "meshes/channel_junction/mesh_2d_coarse.msh")
    # os.path.join(main_dir, "meshes/channel_junction/mesh_2d_fine.msh")
)

mesh = convert_mesh_to_jax(mesh)
class SMESolver(HyperbolicSolver):
    def update_qaux(self, Q, Qaux, Qold, Qauxold, mesh, model, parameters, time, dt):
        dudx = compute_derivatives(Q[1]/Q[0], mesh, derivatives_multi_index=[[0, 0]])[:,0]
        dvdy = compute_derivatives(Q[1+offset]/Q[0], mesh, derivatives_multi_index=[[0, 1]])[:,0]
        Qaux = Qaux.at[0].set(dudx)
        Qaux = Qaux.at[1].set(dvdy)
        return Qaux
solver = SMESolver(settings)

## Solve

In [ ]:
Qnew, Qaux = solver.solve(mesh, model)

2025-08-07 07:56:11.526 | WARNING  | library.misc.io:write_dict_to_hdf5:51 - Skipping unsupported type for key: num_flux -> <class 'function'>
2025-08-07 07:56:11.527 | WARNING  | library.misc.io:write_dict_to_hdf5:51 - Skipping unsupported type for key: nc_flux -> <class 'function'>
2025-08-07 07:56:13.206 | INFO     | library.fvm.solver:log_callback_hyperbolic:37 - iteration: 1, time: 0.003386, dt: 0.003386, next write at time: 0.111111
2025-08-07 07:56:13.244 | INFO     | library.fvm.solver:log_callback_hyperbolic:37 - iteration: 2, time: 0.003652, dt: 0.000266, next write at time: 0.111111
2025-08-07 07:56:13.291 | INFO     | library.fvm.solver:log_callback_hyperbolic:37 - iteration: 3, time: 0.003918, dt: 0.000266, next write at time: 0.111111
2025-08-07 07:56:13.328 | INFO     | library.fvm.solver:log_callback_hyperbolic:37 - iteration: 4, time: 0.004184, dt: 0.000266, next write at time: 0.111111
2025-08-07 07:56:13.389 | INFO     | library.fvm.solver:log_callback_hyperbolic:37 

Abort is in progress...hit ctrl-c again within 5 seconds to forcibly terminate



2025-08-07 07:58:40.626 | INFO     | library.fvm.solver:log_callback_hyperbolic:37 - iteration: 2349, time: 0.679224, dt: 0.000288, next write at time: 0.777778
2025-08-07 07:58:40.677 | INFO     | library.fvm.solver:log_callback_hyperbolic:37 - iteration: 2350, time: 0.679512, dt: 0.000288, next write at time: 0.777778
2025-08-07 07:58:40.733 | INFO     | library.fvm.solver:log_callback_hyperbolic:37 - iteration: 2351, time: 0.679800, dt: 0.000288, next write at time: 0.777778
2025-08-07 07:58:40.798 | INFO     | library.fvm.solver:log_callback_hyperbolic:37 - iteration: 2352, time: 0.680087, dt: 0.000287, next write at time: 0.777778
2025-08-07 07:58:40.846 | INFO     | library.fvm.solver:log_callback_hyperbolic:37 - iteration: 2353, time: 0.680374, dt: 0.000287, next write at time: 0.777778
2025-08-07 07:58:40.905 | INFO     | library.fvm.solver:log_callback_hyperbolic:37 - iteration: 2354, time: 0.680662, dt: 0.000287, next write at time: 0.777778
2025-08-07 07:58:40.956 | INFO    

## Visualization

In [ ]:
io.generate_vtk(os.path.join(settings.output.directory, f"{settings.output.filename}.h5"))
postprocessing.vtk_interpolate_3d(model, settings, Nz=20, filename='out_3d')

In [ ]:
visu.pyvista_3d(settings.output.directory, scale=1.0)

BokehModel(combine_events=True, render_bundle={'docs_json': {'504a6a38-6799-4442-aef5-e9ef5f75b908': {'version…